In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/wns-hack-v1/__results__.html
/kaggle/input/wns-hack-v1/trainWithLogMergeApprox.csv
/kaggle/input/wns-hack-v1/__notebook__.ipynb
/kaggle/input/wns-hack-v1/__output__.json
/kaggle/input/wns-hack-v1/custom.css
/kaggle/input/wns-hack-v1/testWithLogMergeApprox.csv
/kaggle/input/av-wns-hack/train_na17sgz/view_log.csv
/kaggle/input/av-wns-hack/train_na17sgz/train.csv
/kaggle/input/av-wns-hack/train_na17sgz/item_data.csv
/kaggle/input/av-wns-hack/test_aq1fgdb/test.csv
/kaggle/input/av-wns-hack/sample_submission_ipsblct/sample_submission.csv


In [2]:
train = pd.read_csv('../input/wns-hack-v1/trainWithLogMergeApprox.csv')
test = pd.read_csv('../input/wns-hack-v1/testWithLogMergeApprox.csv')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (26,28,30,32,34,36,38,40,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (27,29,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
import pandas as pd
import numpy as np
import gc

import mlcrate as mlc
import pickle as pkl

from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Dense, Flatten, Concatenate, Dot, Reshape, Add, Subtract
from keras import objectives
from keras import backend as K
from keras import regularizers 
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2

Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [4]:
import numpy as np
import tensorflow as tf
import random as rn

# The below is necessary in Python 3.2.3 onwards to
# have reproducible behavior for certain hash-based operations.
# See these references for further details:
# https://docs.python.org/3.4/using/cmdline.html#envvar-PYTHONHASHSEED
# https://github.com/fchollet/keras/issues/2280#issuecomment-306959926

import os

def init_seeds(seed):
    os.environ['PYTHONHASHSEED'] = '0'

    # The below is necessary for starting Numpy generated random numbers
    # in a well-defined initial state.

    np.random.seed(seed)

    # The below is necessary for starting core Python generated random numbers
    # in a well-defined state.

    rn.seed(seed)

    # Force TensorFlow to use single thread.
    # Multiple threads are a potential source of
    # non-reproducible results.
    # For further details, see: https://stackoverflow.com/questions/42022950/which-seeds-have-to-be-set-where-to-realize-100-reproducibility-of-training-res

    session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

    from keras import backend as K

    # The below tf.set_random_seed() will make random number generation
    # in the TensorFlow backend have a well-defined initial state.
    # For further details, see: https://www.tensorflow.org/api_docs/python/tf/set_random_seed

    tf.set_random_seed(seed)

    sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    K.set_session(sess)
    return sess

In [5]:
pd.set_option('display.max_columns',500)
train.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click,impression_time_day,impression_time_minute,impression_time_second,LogPresent_24_hour_before,Count_LogPresent_24_hour_before,LogPresent_192_hour_before,Count_LogPresent_192_hour_before,LogPresent_288_hour_before,Count_LogPresent_288_hour_before,NearestLog_server_time,NearestLog_user_id,NearestLog_device_type_0,NearestLog_item_id_0,NearestLog_device_type_1,NearestLog_item_id_1,NearestLog_device_type_2,NearestLog_item_id_2,NearestLog_device_type_3,NearestLog_item_id_3,NearestLog_device_type_4,NearestLog_item_id_4,NearestLog_device_type_5,NearestLog_item_id_5,NearestLog_device_type_6,NearestLog_item_id_6,NearestLog_device_type_7,NearestLog_item_id_7,NearestLog_device_type_8,NearestLog_item_id_8,NearestLog_device_type_9,NearestLog_item_id_9,NearestLog_device_type_10,NearestLog_item_id_10,NearestLog_device_type_11,NearestLog_item_id_11,NearestLog_device_type_12,NearestLog_item_id_12,NearestLog_device_type_13,NearestLog_item_id_13
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,0,15,0,11,0,0,0,0,0,0,2018-10-19 18:11:00,87862,android,43886.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,1,15,1,11,0,0,0,0,0,0,2018-10-25 05:03:00,63410,android,43209.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,0,15,2,11,0,0,0,0,0,0,2018-10-23 12:59:00,71748,android,122348.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,0,15,2,11,1,8,1,11,1,11,2018-11-14 19:01:00,69209,android,1528.0,android,2715.0,android,104035.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,0,15,2,11,1,2,1,7,1,7,2018-11-14 21:42:00,62873,android,4069.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
v = ['NearestLog_device_type_0', 'app_code','NearestLog_item_id_0','impression_id']
data = pd.concat([train[v],test[v]],0)
data.head()

,NearestLog_device_type_0,app_code,NearestLog_item_id_0,impression_id
0,android,422,43886.0,c4ca4238a0b923820dcc509a6f75849b
1,android,467,43209.0,45c48cce2e2d7fbdea1afc51c7c6ad26
2,android,259,122348.0,70efdf2ec9b086079795c442636b55fb
3,android,244,1528.0,8e296a067a37563370ded05f5a3bf3ec
4,android,473,4069.0,182be0c5cdcd5072bb1864cdee4d3d6e


In [7]:
from sklearn.preprocessing import LabelEncoder

le1 = LabelEncoder()
le1.fit(data['NearestLog_device_type_0'].astype('str'))
data['NearestLog_device_type_0_le']=le1.transform(data['NearestLog_device_type_0'].astype('str'))

In [8]:
features = ['NearestLog_device_type_0_le', 'app_code','NearestLog_item_id_0']
f_size  = [int(data[f].max()) + 1 for f in features]
f_size

[4, 523, 132863]

In [9]:
X = data.groupby(features)['impression_id'].count()
X.head()

NearestLog_device_type_0_le  app_code  NearestLog_item_id_0
0                            0         321.0                   1
                                       1739.0                  1
                                       7992.0                  2
                                       9714.0                  1
                                       12208.0                 1
Name: impression_id, dtype: int64

In [10]:
X = X.unstack().fillna(0)
X.head()

NearestLog_item_id_0                  8.0       10.0      21.0      24.0      \
NearestLog_device_type_0_le app_code                                           
0                           0              0.0       0.0       0.0       0.0   
                            1              0.0       0.0       0.0       0.0   
                            2              0.0       0.0       0.0       0.0   
                            3              0.0       0.0       0.0       0.0   
                            4              0.0       0.0       0.0       0.0   

NearestLog_item_id_0                  25.0      29.0      36.0      46.0      \
NearestLog_device_type_0_le app_code                                           
0                           0              0.0       0.0       0.0       0.0   
                            1              0.0       0.0       0.0       0.0   
                            2              0.0       0.0       0.0       0.0   
                            3              0.0       0.0       0.0       0.0   
                            4              0.0       0.0       0.0       0.0   

NearestLog_item_id_0                  47.0      58.0      65.0      70.0      \
NearestLog_device_type_0_le app_code                                           
0                           0              0.0       0.0       0.0       0.0   
                            1              0.0       0.0       0.0       0.0   
                            2              0.0       0.0       0.0       0.0   
                            3              0.0       0.0       0.0       0.0   
                            4              0.0       0.0       0.0       0.0   

NearestLog_item_id_0                  71.0      74.0      76.0      110.0     \
NearestLog_device_type_0_le app_code                                           
0                           0              0.0       0.0       0.0       0.0   
                            1              0.0       0.0       0.0       0.0   
                            2              0.0       0.0       0.0       0.0   
                            3             21.0       0.0       0.0       0.0   
                            4              0.0       0.0       0.0       0.0   

NearestLog_item_id_0                  127.0     142.0     146.0     147.0     \
NearestLog_device_type_0_le app_code                                           
0                           0              0.0       0.0       0.0       0.0   
                            1              0.0       0.0       0.0       0.0   
                            2              0.0       0.0       0.0       0.0   
                            3              0.0       0.0       1.0       1.0   
                            4              0.0       0.0       0.0       0.0   

NearestLog_item_id_0                  156.0     176.0     177.0     181.0     \
NearestLog_device_type_0_le app_code                                           
0                           0              0.0       0.0       0.0       0.0   
                            1              0.0       0.0       0.0       0.0   
                            2              0.0       0.0       0.0       0.0   
                            3              0.0       4.0       0.0       0.0   
                            4              0.0       0.0       0.0       0.0   

NearestLog_item_id_0                  197.0     201.0     204.0     216.0     \
NearestLog_device_type_0_le app_code                                           
0                           0              0.0       0.0       0.0       0.0   
                            1              0.0       0.0       0.0       0.0   
                            2              0.0       0.0       0.0       0.0   
                            3              0.0       0.0       0.0       0.0   
                            4              0.0       0.0       0.0       0.0   

NearestLog_item_id_0                  243.0     247.0     258.0     267.0

In [11]:
X = X.stack().astype('float32')
X.head()

NearestLog_device_type_0_le  app_code  NearestLog_item_id_0
0                            0         8.0                     0.0
                                       10.0                    0.0
                                       21.0                    0.0
                                       24.0                    0.0
                                       25.0                    0.0
dtype: float32

In [12]:
X = np.log1p(X).reset_index()
X.head()

,NearestLog_device_type_0_le,app_code,NearestLog_item_id_0,0
0,0,0,8.0,0.0
1,0,0,10.0,0.0
2,0,0,21.0,0.0
3,0,0,24.0,0.0
4,0,0,25.0,0.0


In [13]:
X.columns=features + ['num']
X.head()

,NearestLog_device_type_0_le,app_code,NearestLog_item_id_0,num
0,0,0,8.0,0.0
1,0,0,10.0,0.0
2,0,0,21.0,0.0
3,0,0,24.0,0.0
4,0,0,25.0,0.0


In [14]:
X_train = [X[f].values for f in features]
y_train = (X[['num']].values).astype('float32')

In [15]:
(X.num > 0).mean()
w_train = (50 * (y_train > 0).astype('float32') + 1).ravel()

In [16]:
k_latent = 2
embedding_reg = 0.0002
kernel_reg = 0.1

def get_embed(x_input, x_size, k_latent):
    if x_size > 0: #category
        embed = Embedding(x_size, k_latent, input_length=1, 
                          embeddings_regularizer=l2(embedding_reg))(x_input)
        embed = Flatten()(embed)
    else:
        embed = Dense(k_latent, kernel_regularizer=l2(embedding_reg))(x_input)
    return embed

def build_model_1(X, f_size):
    dim_input = len(f_size)
    
    input_x = [Input(shape=(1,)) for i in range(dim_input)] 
     
    biases = [get_embed(x, size, 1) for (x, size) in zip(input_x, f_size)]
    
    factors = [get_embed(x, size, k_latent) for (x, size) in zip(input_x, f_size)]
    
    s = Add()(factors)
    
    diffs = [Subtract()([s, x]) for x in factors]
    
    dots = [Dot(axes=1)([d, x]) for d,x in zip(diffs, factors)]
    
    x = Concatenate()(biases + dots)
    x = BatchNormalization()(x)
    output = Dense(1, activation='relu', kernel_regularizer=l2(kernel_reg))(x)
    model = Model(inputs=input_x, outputs=[output])
    opt = Adam(clipnorm=0.5)
    model.compile(optimizer=opt, loss='mean_squared_error')
    output_f = factors + biases
    model_features = Model(inputs=input_x, outputs=output_f)
    return model, model_features

In [17]:
model, model_features = build_model_1(X_train, f_size)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 1, 2)         8           input_1[0][0]                    
__________________________________________________________________________________________________
embedding_

In [18]:
model_features.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 1, 2)         8           input_1[0][0]                    
__________________________________________________________________________________________________
embedding_

In [19]:
n_epochs = 100
P = 17
try:
    del sess
except:
    pass
sess = init_seeds(0)

batch_size = 2**P
print(batch_size)
model, model_features = build_model_1(X_train, f_size)
earlystopper = EarlyStopping(patience=0, verbose=1)

model.fit(X_train,  y_train, 
          epochs=n_epochs, batch_size=batch_size, verbose=1, shuffle=True, 
          validation_data=(X_train, y_train), 
          sample_weight=w_train,
          callbacks=[earlystopper],
         )

131072
Train on 10823448 samples, validate on 10823448 samples
Epoch 1/100
10823448/10823448 [==============================] - 13s 1us/step - loss: 0.5761 - val_loss: 0.1826
Epoch 2/100
10823448/10823448 [==============================] - 11s 1us/step - loss: 0.3125 - val_loss: 0.1351
Epoch 3/100
10823448/10823448 [==============================] - 11s 1us/step - loss: 0.2703 - val_loss: 0.1159
Epoch 4/100
10823448/10823448 [==============================] - 11s 1us/step - loss: 0.2489 - val_loss: 0.1042
Epoch 5/100
10823448/10823448 [==============================] - 11s 1us/step - loss: 0.2347 - val_loss: 0.0922
Epoch 6/100
10823448/10823448 [==============================] - 11s 1us/step - loss: 0.2246 - val_loss: 0.0865
Epoch 7/100
10823448/10823448 [==============================] - 11s 1us/step - loss: 0.2171 - val_loss: 0.0783
Epoch 8/100
10823448/10823448 [==============================] - 11s 1us/step - loss: 0.2115 - val_loss: 0.0747
Epoch 9/100
10823448/10823448 [==========

In [20]:
X_pred = model_features.predict(X_train, batch_size=batch_size)
len(X_pred)

6

In [21]:
factors = X_pred[:len(features)]

biases = X_pred[len(features):2*len(features)]

for f, X_p in zip(features, factors):
    for i in range(k_latent):
        X['%s_fm_factor_%d' % (f, i)] = X_p[:,i]

for f, X_p in zip(features, biases):
    X['%s_fm_bias' % (f)] = X_p[:,0]

X.head()

,NearestLog_device_type_0_le,app_code,NearestLog_item_id_0,num,NearestLog_device_type_0_le_fm_factor_0,NearestLog_device_type_0_le_fm_factor_1,app_code_fm_factor_0,app_code_fm_factor_1,NearestLog_item_id_0_fm_factor_0,NearestLog_item_id_0_fm_factor_1,NearestLog_device_type_0_le_fm_bias,app_code_fm_bias,NearestLog_item_id_0_fm_bias
0,0,0,8.0,0.0,-0.377059,-0.416286,-0.050631,-0.01794,-0.023627,-0.025817,-0.049886,0.033474,-0.054376
1,0,0,10.0,0.0,-0.377059,-0.416286,-0.050631,-0.01794,-0.023861,-0.026094,-0.049886,0.033474,-0.054424
2,0,0,21.0,0.0,-0.377059,-0.416286,-0.050631,-0.01794,-0.005918,-0.006949,-0.049886,0.033474,-0.025934
3,0,0,24.0,0.0,-0.377059,-0.416286,-0.050631,-0.01794,-0.019685,-0.022096,-0.049886,0.033474,-0.050691
4,0,0,25.0,0.0,-0.377059,-0.416286,-0.050631,-0.01794,-0.002610,-0.004150,-0.049886,0.033474,-0.018412


In [22]:
#X['os_version_le'] = np.array(le.inverse_transform(np.array(X['os_version_le'])))
X['NearestLog_device_type_0_le'] = np.array(le1.inverse_transform(np.array(X['NearestLog_device_type_0_le'])))

In [23]:
X.to_csv('FM_feats.csv',index=False)

In [24]:
## Fin ##